##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Кастомные слои

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/customization/custom_layers"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/customization/custom_layers.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

Мы рекомендуем использовать `tf.keras` в качестве высокоуровневого API для построения нейронных сетей. Тем не менее, вы можете использовать большинство API-интерфейсов TensorFlow и они поддерживают быстроые вычисления(eager execution).

In [ ]:
import tensorflow as tf

In [ ]:
print(tf.test.is_gpu_available())

## Слои: общие установки полезных операций

В большинстве случаев при написании кода для моделей машинного обучения вы хотите работать на более высоком уровне абстракции, а не разбираясь с отдельными методами и отдельными переменными.

Многие модели машинного обучения можно описать как композицию и последовательное сложение относительно простых слоев. TensorFlow предоставляет как готовые наборы из множества общих слоев, так и простые способы написания собственных слоев для конкретного приложения, либо с нуля, либо в виде создания композиции из существующих слоев.

TensorFlow включает полный API [Keras](https://keras.io) в пакете tf.keras, а слои Keras будут очень полезны при построении ваших собственных моделей.


In [ ]:
# В пакете tf.keras.layers слои являются объектами. Чтобы построить слой,
# просто создайте объект класса слоя. Большинство слоев принимают в качестве первого аргумента число
# выходной размерности/каналов.
layer = tf.keras.layers.Dense(100)
# Размерность входных данных часто не нужна, так как ее можно передать
# при первом использовании слоя, но вы можете
# указать ее вручную, что полезно в некоторых сложных моделях.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

Полный список уже существующих слоев можно увидеть в [документации](https://www.tensorflow.org/api_docs/python/tf/keras/layers). Он включает Dense(полносвязанный слой),
Conv2D, LSTM, BatchNormalization, Dropout и многие другие.

In [ ]:
# Для использования слоя просто вызовите его, передав данные
layer(tf.zeros([10, 5]))

In [ ]:
# Слои имеют много полезных методов. Например, вы можете проверить все переменные
# в слое вызвав `layer.variables` или просмотреть список обучаемых переменных с помощью
# `layer.trainable_variables`. В нашем случае полносвязанный слой
# будут иметь переменные для весов и смещений.
layer.variables

In [ ]:
# Переменные также доступны через приятные акцессоры
layer.kernel, layer.bias

## Реализация пользовательских слоев

Лучший способ реализовать свой собственный слой - это расширить класс tf.keras.Layer и реализовать:

1. `__init__`, где вы можете выполнять инициализацию, независимую от входных данных
2. `build`, где вы знаете размерность входных тензоров и можете выполнить остальную часть инициализации.
3. `call`, где вы выполняете прямое вычисление.

Обратите внимание, что вам не нужно ждать вызова `build` для создания ваших переменных, вы можете создать их прямо в методе `__init__`. Однако преимущество их создания в `build` заключается в том, что вы создаете переменные на основании входных данных, с которыми будет работать слой. С другой стороны, создание переменных в `__init__` будет означать, что размерность входных данных, необходимая для создания переменных, должна быть явно указана.

In [ ]:
class MyDenseLayer(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(MyDenseLayer, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel",
                                  shape=[int(input_shape[-1]),
                                         self.num_outputs])

  def call(self, input):
    return tf.matmul(input, self.kernel)

layer = MyDenseLayer(10)

In [ ]:
_ = layer(tf.zeros([10, 5])) # Вызывается метод layer.builds а затем call

In [ ]:
print([var.name for var in layer.trainable_variables])

Overall code is easier to read and maintain if it uses standard layers whenever possible, as other readers will be familiar with the behavior of standard layers. If you want to use a layer which is not present in `tf.keras.layers`, consider filing a [github issue](http://github.com/tensorflow/tensorflow/issues/new) or, even better, sending us a pull request!

Любой код легче читать и поддерживать, если он по возможности использует стандартные слои, так как другие читатели наверняка будут знакомы с поведением стандартных слоев. Если вы хотите использовать слой, которого нет в `tf.keras.layers`, подумайте о том, чтобы создать [багрепорт на github](http://github.com/tensorflow/tensorflow/issues/new) или, что еще лучше, отправьте нам запрос на добавление вашего слоя!

## Модели: Составление слоев

Многие интересные вещи, например сложные слои, в моделях машинного обучения реализованы путем компоновки существующих слоев. Например, каждый остаточный блок в сети ResNet представляет собой комбинацию сверток, пакетных нормализаций и ярлыка. Слои могут быть вложены в другие слои.

Обычно, если нам нужны такие методы модели, как:` Model.fit`, `Model.evaluate` и` Model.save` мы наследуемся от `keras.Model`,  (более подробно см. [Пользовательские слои и модели Keras](../../guide/keras/custom_layers_and_models.ipynb)).

Еще одна особенность, предоставляемая `keras.Model` и чего нет в `keras.layers.Layer`, заключается в том, что в дополнение к отслеживанию переменных, `keras.Model` также отслеживает свои внутренние слои, что упрощает их проверку.

Например, вот блок ResNet:

In [ ]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size, filters):
    super(ResnetIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters

    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

In [ ]:
_ = block(tf.zeros([1, 2, 3, 3])) 

In [ ]:
block.layers

In [ ]:
len(block.variables)

In [ ]:
block.summary()

Часто модели, состоящие из множества слоев, просто вызывают один слой за другим. Это можно сделать очень просто, используя `tf.keras.Sequential`:

In [ ]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

In [ ]:
my_seq.summary()

# Что дальше

Теперь вы можете вернуться к предыдущему руководству и адаптировать пример линейной регрессии с использованием слоев и модели.